In [4]:
import hmmlearn.hmm as hmm
import numpy as np
from Bio import SeqIO

# Dziś na wykładzie omówimy ukryte modele Markowa i algorytmy rekonstrukcji parametrów z danych.
#
# Zadania na dziś:

# 0. Zapoznaj się z modułem hmmlearn. Będziemy go wykorzystywać do uczenia modeli z emisjami

# 1. Wyspy CpG znajdują się często w genomach, w szczególności genomie ludzkim. Spróbuj zdefiniować ukryty model
# Markowa, który ma 2 stany i spróbuj nauczyć go na sekwncji zawartej w pliku cpg.fa. Zrób to zarówno dla sekwencji
# liter (ACGT), jak i dla sekwencji dwunukleotydów (AA,AC,AG,AT, itp…) Czy możesz zinterpretować macierze emisji i
# przypisać jeden ze stanów do wysp CpG? Wykonaj kilkakrotnie proces uczenia (Baum-Welch) i zobacz czy wyniki są
# podobne. Jak interpretujesz prawdopodobieństwa w macierzy przejść. Czy coś możesz powiedzieć o średniej długości
# wysp CpG?
#
# Warto przyjrzeć się przykładowi użycia klasy CategoricalHMM


def dna_to_ordinal(dna: str) -> int:
    return {"A": 0, "C": 1, "G": 2, "T": 3}[dna]


full_training_seq = SeqIO.read("data/cpg.fa", "fasta").seq
# full_training_seq = full_training_seq[:1000]    # TODO: remove this line after testing
full_training_seq = [dna_to_ordinal(dna) for dna in full_training_seq]

# Weźmy 8 stanów ukrytych i 4 emisje (A, C, G, T)
# Stany podzielimy na dwie kategorie - nie-CpG i CpG - po cztery stany w każdej kategorii
# Pierwszy kategoria, nie-CpG, będzie miała stany 0, 1, 2, 3
# Druga kategoria, CpG, będzie miała stany 4, 5, 6, 7
# Emisje będą odpowiadały literom A, C, G, T

p_state_switch = 0.01
p4 = (1 - p_state_switch) / 4
ps4 = p_state_switch / 4
p_cg = 0.4
ps4_cg = p_state_switch / 4
p4_cg = (1 - p_state_switch - p_cg) / 3

states_matrix = np.array(
    [
        # (nCpG)  A   C   G   T  | (CpG)  A    C    G    T
        [p4, p4, p4, p4, ps4, ps4, ps4, ps4],
        [p4, p4, p4, p4, ps4, ps4, ps4, ps4],
        [p4, p4, p4, p4, ps4, ps4, ps4, ps4],
        [p4, p4, p4, p4, ps4, ps4, ps4, ps4],
        [ps4, ps4, ps4, ps4, p4, p4, p4, p4],
        [ps4_cg, ps4_cg, ps4_cg, ps4_cg, p4_cg, p4_cg, p_cg, p4_cg],
        [ps4, ps4, ps4, ps4, p4, p4, p4, p4],
        [ps4, ps4, ps4, ps4, p4, p4, p4, p4],
    ]
)

print(states_matrix)

print(states_matrix[dna_to_ordinal("C") + 4, dna_to_ordinal("G") + 4])
print(states_matrix[dna_to_ordinal("C") + 4, dna_to_ordinal("A") + 4])
print(states_matrix[dna_to_ordinal("C") + 4, dna_to_ordinal("C") + 4])
print(states_matrix[dna_to_ordinal("C") + 4, dna_to_ordinal("T") + 4])

emissions_matrix = np.array(
    [
        # A  C  G  T
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)

cpg_dna_hmm = hmm.CategoricalHMM(n_components=8, init_params="")
cpg_dna_hmm.n_features_ = 4
# cpg_dna_hmm.startprob_ = np.array([0.125] * 8)
cpg_dna_hmm.transmat_ = states_matrix
cpg_dna_hmm.emissionprob_ = emissions_matrix


training_reshaped = np.array(full_training_seq).reshape(1, -1)
cpg_dna_hmm.fit(training_reshaped)

[[0.2475     0.2475     0.2475     0.2475     0.0025     0.0025
  0.0025     0.0025    ]
 [0.2475     0.2475     0.2475     0.2475     0.0025     0.0025
  0.0025     0.0025    ]
 [0.2475     0.2475     0.2475     0.2475     0.0025     0.0025
  0.0025     0.0025    ]
 [0.2475     0.2475     0.2475     0.2475     0.0025     0.0025
  0.0025     0.0025    ]
 [0.0025     0.0025     0.0025     0.0025     0.2475     0.2475
  0.2475     0.2475    ]
 [0.0025     0.0025     0.0025     0.0025     0.19666667 0.19666667
  0.4        0.19666667]
 [0.0025     0.0025     0.0025     0.0025     0.2475     0.2475
  0.2475     0.2475    ]
 [0.0025     0.0025     0.0025     0.0025     0.2475     0.2475
  0.2475     0.2475    ]]
0.4
0.19666666666666666
0.19666666666666666
0.19666666666666666


CategoricalHMM(init_params='', n_components=8,
               random_state=RandomState(MT19937) at 0x7F09F4138640)

In [10]:
print(cpg_dna_hmm.transmat_[dna_to_ordinal("C") + 4, dna_to_ordinal("G") + 4])
print(cpg_dna_hmm.transmat_[dna_to_ordinal("C") + 4, dna_to_ordinal("A") + 4])
print(cpg_dna_hmm.transmat_[dna_to_ordinal("C") + 4, dna_to_ordinal("C") + 4])
print(cpg_dna_hmm.transmat_[dna_to_ordinal("C") + 4, dna_to_ordinal("T") + 4])

print(cpg_dna_hmm.transmat_)
print(cpg_dna_hmm.emissionprob_)
print(cpg_dna_hmm.startprob_)
cpg_dna_hmm.startprob_ = np.array([0.125] * 8)

training_reshaped = np.array(full_training_seq[:1000]).reshape(1, -1)
result = cpg_dna_hmm.decode(training_reshaped)
# print(result)

# 2. czasami potrzebujemy użyć łańcuchów Markowa do segmentacji sygnału. Weźmy przykładowy plik – dane o methylacji
# histonów w rzodkiewniku (H3K9me2-crh6-3-chr1.sgr) zawierający poziom metylacji w różnych pozycjach (plik jest
# tekstowy, każda linia zawiera informacje o pozycji i wartośći). Wykorzystaj HMM z emisjami Gaussowskimi do
# segmentacji genomu na 2 lub 3 stany. Jakie są wyestymowane wartości średnie dla różnych stanów? Przykład
# wykorzystania HMMów z emisjami Gaussowskimi można znaleźć tu

# praca domowa (2 pkt):
#
# Wykorzystaj model nauczony na danych o CpG i przetestuj które z 30 sekwencji w pliku cpg_test.fa są naprawdę
# wyspami CpG. Jako wynik proszę przysłać program w pythonie i wynik w pliku tekstowym (w każdej linii możemy podać
# parawdopodobieństwo a'posteriori tego, że dana sekwencja z wejścia pochodzi z modelu CpG).

request_seqs = SeqIO.parse("data/cpg_test.fa", "fasta")


def map_to_cpg_state(state: int) -> int:
    if state in [0, 1, 2, 3]:
        return 0
    elif state in [4, 5, 6, 7]:
        return 1
    else:
        raise ValueError(f"Unknown state: {state}")


def count_cpg(states: [int]) -> int:
    assert all(map(lambda s: s in [0, 1], states))
    return sum(states)


for req_seq in request_seqs:
    req_name = req_seq.description
    req_seq = req_seq.seq
    req_seq = [dna_to_ordinal(dna) for dna in req_seq]
    req_seq = np.array(req_seq).reshape(1, -1)
    result = cpg_dna_hmm.decode(req_seq)
    # print(result)
    res_len = len(result[1])
    cpg_states = [map_to_cpg_state(state) for state in result[1]]
    cpg_count = count_cpg(cpg_states)
    cpg_ratio = cpg_count / res_len
    is_cpg = cpg_ratio > 0.5
    print(
        f"Sequence: {req_name} "
        f"| Ratio: {cpg_ratio:.4f} "
        f"| {'CpG island' if cpg_ratio > 0.5 else 'Not CpG island'}"
    )

0.5529545303516914
0.14098975406995495
0.16504364879096567
0.1409474299427797
[[2.49469529e-01 2.50988774e-01 2.50325351e-01 2.49126343e-01
  1.97036814e-05 2.63165996e-05 1.68420305e-05 2.71402099e-05]
 [2.50677215e-01 2.48491056e-01 2.49747686e-01 2.50951380e-01
  2.21405980e-05 3.30408103e-05 3.44267232e-05 4.30554123e-05]
 [2.50136689e-01 2.50727085e-01 2.49532915e-01 2.49517409e-01
  1.54387358e-05 2.74964644e-05 1.49111180e-05 2.80544125e-05]
 [2.50316410e-01 2.48430818e-01 2.50867609e-01 2.50269806e-01
  3.36010405e-05 4.12268905e-05 2.00577982e-05 2.04709656e-05]
 [2.48141954e-05 2.45455321e-05 1.82200442e-05 2.21799902e-05
  1.99473168e-01 4.00368342e-01 2.00616919e-01 1.99451811e-01]
 [1.12115660e-05 2.06644647e-05 2.04153384e-05 1.23454755e-05
  1.40989754e-01 1.65043649e-01 5.52954530e-01 1.40947430e-01]
 [1.81464705e-05 1.79291775e-05 1.92541236e-05 1.91800192e-05
  2.00119146e-01 3.99935933e-01 2.00081688e-01 1.99788724e-01]
 [1.89536982e-05 3.39956888e-05 2.07920104e-05 

In [11]:
cpg_dna_hmm.startprob_

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])

In [12]:
cpg_dna_hmm.emissionprob_

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [13]:
cpg_dna_hmm.n_features_, cpg_dna_hmm.n_components

(4, 8)